In [1]:
import pandas as pd
import tweepy

In [2]:
# Leer las llaves
keys = pd.read_csv("C:/Users/yayat/OneDrive/Documentos/Jeimmy/Universidad/NPL/Clase del 13-05-2021/mis_llaves.csv", header=None)
keys = dict(zip(keys[0],keys[1]))
keys.keys()

dict_keys(['api_key', 'api_secret_key', 'access_token', 'access_token_secret', 'bearer_token'])

In [3]:
# Autenticar a través de la API 
auth = tweepy.OAuthHandler(keys['api_key'], keys['api_secret_key'])
auth.set_access_token(keys['access_token'], keys['access_token_secret'])
api = tweepy.API(auth, wait_on_rate_limit=True)

In [28]:
# Paso 4 recolectar datos
tweets = tweepy.Cursor(api.search,
                       q=['paro nacional -RT'], # RT es que quite los twits que son retwitiados
                       lang='es',
                       geocode='4.7110,-74.0721,100km', # es una cadena de texto que permite buscar en ese punto las cordenadas
                       tweet_mode='extended',# la cantidad de carqacteres extendida
                       include_rts=False, # para no incluir retwites
                       ).items(100) # el numero de twits a buscar

In [29]:
t = [tweet for tweet in tweets]

In [30]:
t_processed = []

for tweet in t: 
    temp = []
    temp.append(tweet.full_text)
    temp.append(tweet.created_at)
    temp.append('Bogotá')
    temp.append(tweet.user.location)
    temp.append(tweet.id)
    temp.append(f"https://twitter.com/i/web/stuatus/{tweet.id}")
    
    t_processed.append(temp)
    
    
data = pd.DataFrame(t_processed)
data.columns = ['tweets', 'date', 'city', 'user_location', 'id', 'link']
data.head()


,tweets,date,city,user_location,id,link
0,Ya está disponible el podcast que grabé junto ...,2021-05-17 20:11:49,Bogotá,"Bogotá, Colombia.",1394385185781731333,https://twitter.com/i/web/stuatus/139438518578...
1,“Nos quitaron un ojo pero seguimos teniendo nu...,2021-05-17 20:05:00,Bogotá,"Bogotá, Colombia",1394383469938024450,https://twitter.com/i/web/stuatus/139438346993...
2,#17May Convocatorias del denominado Comité de ...,2021-05-17 20:04:42,Bogotá,"Bogotá, Colombia",1394383397187989507,https://twitter.com/i/web/stuatus/139438339718...
3,Una nueva agenda nacional: análisis del excomi...,2021-05-17 20:03:31,Bogotá,Bogotá,1394383098171826182,https://twitter.com/i/web/stuatus/139438309817...
4,Desde el Semillero 𝐃𝐢𝐠𝐧𝐢𝐝𝐚𝐝𝐞𝐬 𝐞𝐧 𝐭𝐞𝐧𝐬𝐢𝐨́𝐧 y en...,2021-05-17 20:02:18,Bogotá,Bogotá D.C.,1394382790897061890,https://twitter.com/i/web/stuatus/139438279089...


In [31]:
data = data[['tweets', 'date']]
data.head()

,tweets,date
0,Ya está disponible el podcast que grabé junto ...,2021-05-17 20:11:49
1,“Nos quitaron un ojo pero seguimos teniendo nu...,2021-05-17 20:05:00
2,#17May Convocatorias del denominado Comité de ...,2021-05-17 20:04:42
3,Una nueva agenda nacional: análisis del excomi...,2021-05-17 20:03:31
4,Desde el Semillero 𝐃𝐢𝐠𝐧𝐢𝐝𝐚𝐝𝐞𝐬 𝐞𝐧 𝐭𝐞𝐧𝐬𝐢𝐨́𝐧 y en...,2021-05-17 20:02:18


In [32]:
import re 

pattern1 = '?P<pic>pic.twitter.com/[^\s]+'
pattern2 = '?P<url>https?://[^\s]+'

In [33]:
def text_clean(row):
    text = row['tweets']
    
    links = [tuple(j for j in i if j)[-1] for i in re.findall(f"({pattern1})|({pattern2})",text)]
    for link in links:
        text = text.replace(link,"")
    
    hashtags = [interaction for interaction in text.split() if interaction.startswith("#")]
    for hashtag in hashtags:
        text = text.replace(hashtag, "")
        
    mentions = [interaction for interaction in text.split() if interaction.startswith("@")]
    for mention in mentions:
        text = text.replace(mention, "")
        
    return text, links, hashtags, mentions

data[['texto_limpio', 'links', 'hashtags', 'mentions']] = data.apply(text_clean, axis=1, result_type='expand')
data

,tweets,date,texto_limpio,links,hashtags,mentions
0,Ya está disponible el podcast que grabé junto ...,2021-05-17 20:11:49,Ya está disponible el podcast que grabé junto ...,[https://t.co/wqt3ytFU8A],[],[@ArielAnaliza]
1,“Nos quitaron un ojo pero seguimos teniendo nu...,2021-05-17 20:05:00,“Nos quitaron un ojo pero seguimos teniendo nu...,"[https://t.co/Dpi4YupRrp, https://t.co/QvBRkaq...",[],[]
2,#17May Convocatorias del denominado Comité de ...,2021-05-17 20:04:42,Convocatorias del denominado Comité de Paro N...,[https://t.co/x0SXMsKdOd],[#17May],[]
3,Una nueva agenda nacional: análisis del excomi...,2021-05-17 20:03:31,Una nueva agenda nacional: análisis del excomi...,[https://t.co/BC1xBPNq2g],[],[]
4,Desde el Semillero 𝐃𝐢𝐠𝐧𝐢𝐝𝐚𝐝𝐞𝐬 𝐞𝐧 𝐭𝐞𝐧𝐬𝐢𝐨́𝐧 y en...,2021-05-17 20:02:18,Desde el Semillero 𝐃𝐢𝐠𝐧𝐢𝐝𝐚𝐝𝐞𝐬 𝐞𝐧 𝐭𝐞𝐧𝐬𝐢𝐨́𝐧 y en...,"[https://t.co/Pb9kPeMJYV., https://t.co/vmMLS3...",[],[]
...,...,...,...,...,...,...
95,Conozca todos los detalles sobre las marchas y...,2021-05-17 17:03:30,Conozca todos los detalles sobre las marchas y...,"[https://t.co/Cj7oQt6jeE, https://t.co/HS9qqF5...",[],[]
96,#NoticiasDeLaHora 918 policías han resultado h...,2021-05-17 17:01:38,918 policías han resultado heridos en los 18 ...,"[https://t.co/LPPvT06WLI, https://t.co/kEglUMU...",[#NoticiasDeLaHora],[]
97,¡EN VIVO!\nLa @ComisiondePaz del @SenadoGovCo ...,2021-05-17 17:01:32,¡EN VIVO!\nLa del está hoy en Pereira escuch...,[https://t.co/bc1eHb4Hef],[],"[@ComisiondePaz, @SenadoGovCo]"
98,#NoticiasDeLaHora Hoy nuevamente el Gobierno N...,2021-05-17 17:00:36,Hoy nuevamente el Gobierno Nacional y el Comi...,"[https://t.co/LPPvT06WLI, https://t.co/cQxXcD7...",[#NoticiasDeLaHora],[]


In [34]:
import regex
import emoji

def get_emojis(text):
    emoji_list = []
    data = regex.findall(r'\X', text)
    for word in data:
        if any(char in emoji.UNICODE_EMOJI['en'] for char in word):
            emoji_list.append(word)

    return emoji_list
  
  

data['emojis'] = data['texto_limpio'].apply(lambda text: get_emojis(text))
data.head()

,tweets,date,texto_limpio,links,hashtags,mentions,emojis
0,Ya está disponible el podcast que grabé junto ...,2021-05-17 20:11:49,Ya está disponible el podcast que grabé junto ...,[https://t.co/wqt3ytFU8A],[],[@ArielAnaliza],[]
1,“Nos quitaron un ojo pero seguimos teniendo nu...,2021-05-17 20:05:00,“Nos quitaron un ojo pero seguimos teniendo nu...,"[https://t.co/Dpi4YupRrp, https://t.co/QvBRkaq...",[],[],[]
2,#17May Convocatorias del denominado Comité de ...,2021-05-17 20:04:42,Convocatorias del denominado Comité de Paro N...,[https://t.co/x0SXMsKdOd],[#17May],[],[]
3,Una nueva agenda nacional: análisis del excomi...,2021-05-17 20:03:31,Una nueva agenda nacional: análisis del excomi...,[https://t.co/BC1xBPNq2g],[],[],[]
4,Desde el Semillero 𝐃𝐢𝐠𝐧𝐢𝐝𝐚𝐝𝐞𝐬 𝐞𝐧 𝐭𝐞𝐧𝐬𝐢𝐨́𝐧 y en...,2021-05-17 20:02:18,Desde el Semillero 𝐃𝐢𝐠𝐧𝐢𝐝𝐚𝐝𝐞𝐬 𝐞𝐧 𝐭𝐞𝐧𝐬𝐢𝐨́𝐧 y en...,"[https://t.co/Pb9kPeMJYV., https://t.co/vmMLS3...",[],[],[]


In [35]:
data['hora'] = data['date'].dt.floor('T').dt.time
temp = pd.DataFrame(data.hora.value_counts()).reset_index()
temp.columns = ['hora', 'cnt']
temp = temp.sort_values('hora')
temp


,hora,cnt
58,16:59:00,1
67,17:00:00,1
6,17:01:00,2
44,17:03:00,1
42,17:05:00,1
...,...,...
71,20:02:00,1
60,20:03:00,1
66,20:04:00,1
70,20:05:00,1


In [36]:
import plotly.io as pio
import plotly.express as px
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

In [37]:
import matplotlib.pyplot as plt
%matplotlib inline
init_notebook_mode(connected=True)

In [40]:
trace = go.Scatter(x=temp.hora.values,
                   y=temp.cnt.values,
                   text=[f"Fecha: {f}<br>Tuits: {c}" for f,c in zip(temp.hora.values,temp.cnt.values)],
                   hoverinfo='text',
                   mode='lines+markers',
                   name='Horas',
                   line={'color': 'purple'})

layout = go.Layout(title="Número de tuits por hora")

fig = go.Figure(data=[trace], layout=layout)
iplot(fig)